In [1]:
import http.server
import socketserver
import os
import hashlib
from datetime import datetime
import time
import threading

PORT = 8000
FILE_TO_SERVE = "index.html"

class CachingHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/':
            self.path = FILE_TO_SERVE
        try:
            file_path = self.translate_path(self.path)
            last_modified_time = os.path.getmtime(file_path)
            last_modified_header = self.date_time_string(last_modified_time)

            with open(file_path, 'rb') as f:
                content = f.read()

            etag = f'"{hashlib.md5(content).hexdigest()}"'

            client_etag = self.headers.get('If-None-Match')
            client_last_modified = self.headers.get('If-Modified-Since')

            if client_etag and client_etag == etag:
                self.send_response(304, "Not Modified")
                self.end_headers()
                return

            if client_last_modified:
                try:
                    client_modified_time = time.mktime(
                        time.strptime(client_last_modified, "%a, %d %b %Y %H:%M:%S %Z")
                    )
                    if last_modified_time <= client_modified_time:
                        self.send_response(304, "Not Modified")
                        self.end_headers()
                        return
                except (ValueError, TypeError):
                    pass

            self.send_response(200, "OK")
            self.send_header("Content-type", "text/html")
            self.send_header("Content-Length", str(len(content)))
            self.send_header("Last-Modified", last_modified_header)
            self.send_header("ETag", etag)
            self.end_headers()
            self.wfile.write(content)

        except FileNotFoundError:
            self.send_error(404, f"File Not Found: {self.path}")

if not os.path.exists(FILE_TO_SERVE):
    with open(FILE_TO_SERVE, 'w') as f:
        f.write("<!DOCTYPE html>\n<html>\n<head><title>Test Page</title></head>\n")
        f.write("<body><h1>Hello, World! This is the first version.</h1></body>\n</html>\n")

Handler = CachingHTTPRequestHandler
httpd = socketserver.TCPServer(("", PORT), Handler)

def run_server():
    print(f"Serving at port {PORT}")
    print("Open your browser to http://localhost:8000")
    httpd.serve_forever()

thread = threading.Thread(target=run_server, daemon=True)
thread.start()


Serving at port 8000
Open your browser to http://localhost:8000


In [2]:
import socket
import threading

HOST, PORT = '', 8080

def create_response(body, cookie=None):
    response = "HTTP/1.1 200 OK\r\n"
    response += "Content-Type: text/html\r\n"
    response += f"Content-Length: {len(body)}\r\n"
    if cookie:
        response += f"Set-Cookie: session_id={cookie}\r\n"
    response += "\r\n"
    response += body
    return response

def parse_cookies(request_headers):
    cookies = {}
    for header in request_headers:
        if header.lower().startswith("cookie:"):
            cookie_data = header.split(':', 1)[1].strip()
            pairs = cookie_data.split(';')
            for pair in pairs:
                if '=' in pair:
                    key, value = pair.split('=', 1)
                    cookies[key.strip()] = value.strip()
    return cookies

def handle_request(client_socket):
    request_data = client_socket.recv(1024).decode('utf-8')
    if not request_data:
        return
    
    headers = request_data.split('\r\n')
    print("--- Received Request ---")
    print(request_data)
    print("------------------------")

    cookies = parse_cookies(headers)
    
    if 'session_id' in cookies:
        user_id = cookies['session_id']
        body = f"<h1>Welcome back, {user_id}!</h1><p>Your session is maintained.</p>"
        response = create_response(body)
    else:
        user_id = "User123"
        body = "<h1>Welcome, new visitor!</h1><p>A cookie has been set for your session.</p>"
        response = create_response(body, cookie=user_id)
    
    client_socket.sendall(response.encode('utf-8'))
    client_socket.close()

def run_server():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        server_socket.bind((HOST, PORT))
        server_socket.listen(1)
        print(f"Listening on port {PORT}...")
        print(f"Open your browser to http://localhost:{PORT}")
        
        while True:
            client_socket, addr = server_socket.accept()
            print(f"Accepted connection from {addr}")
            handle_request(client_socket)

# Launch server in a background thread for Jupyter
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()


Listening on port 8080...
Open your browser to http://localhost:8080
